# Visualization: Trading Session

In [9]:
!py -m pip install pandas matplotlib numpy seaborn altair coloredlogs scikit-learn keras tensorflow

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.4/10.6 MB 23.9 MB/s eta 0:00:01
   --- ------------------------------------ 0.8/10.6 MB 13.2 MB/s eta 0:00:01
   ----- ---------------------------------- 1.5/10.6 MB 13.5 MB/s eta 0:00:01
   -------- ------------------------------- 2.2/10.6 MB 14.0 MB/s eta 0:00:01
   ---------- ----------------------------- 2.8/10.6 MB 13.7 MB/s eta 0:00:01
   ---------- ----------------------------- 2.9/10.6 MB 11.6 MB/s eta 0:00:01
   ------------ --------------------------- 3.3/10.6 MB 11.9 MB/s eta 0:00:01
   -------------- ------------------------- 3.9/10.6 MB 11.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4.6/10.6 MB 12.2 MB/s eta 0:00:01
   -------------------- ------------------- 5.3/10.6 MB 12.6 MB/s eta 0:00:0

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
import coloredlogs
from trading_bot.agent import Agent
from trading_bot.utils import show_eval_result, switch_k_backend_device
from trading_bot.methods import evaluate_model

In [2]:
import keras
import tensorflow as tf

print("Keras version:", keras.__version__)
print("TensorFlow version:", tf.__version__)


Keras version: 3.1.1
TensorFlow version: 2.16.1


In [3]:
# Load the dataset with real and predicted values
df = pd.read_csv('data/predictions/fs_pred_lstm.csv')

# Convert dates from object to DateTime type
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)

df

C:\Users\Devesh Pansare\AppData\Local\Temp\ipykernel_31432\2523953611.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)


,date,Real,Predicted,Accuracy
0,2021-08-13,515.92,498.88830,NaN
1,2021-08-16,517.92,497.49692,0.0
2,2021-08-17,518.91,497.39900,0.0
3,2021-08-18,521.87,498.64307,1.0
4,2021-08-19,543.71,501.52496,1.0
...,...,...,...,...
222,2022-07-01,179.95,188.05637,0.0
223,2022-07-05,185.88,185.29272,0.0
224,2022-07-06,184.06,185.92947,0.0
225,2022-07-07,189.27,186.73022,1.0


In [1]:
import pandas as pd
import altair as alt
import coloredlogs
from trading_bot.agent import Agent
from trading_bot.methods import evaluate_model, count_action
from trading_bot.utils import show_eval_result, switch_k_backend_device

import pandas as pd
import altair as alt

def visualize(df, history, title="Trading Session"):
    # Ensure 'date' column is a datetime type for proper charting
    df['date'] = pd.to_datetime(df['date'])

    # Create a line chart for the real prices
    price_chart = alt.Chart(df).mark_line().encode(
        x='date:T',
        y=alt.Y('Real:Q', title='Price'),
        color=alt.value('green')  # Line color, can be adjusted as needed
    )

    # Convert history into a DataFrame
    history_df = pd.DataFrame(history, columns=['date', 'price', 'action', 'quantity'])
    history_df['date'] = pd.to_datetime(history_df['date'])  # convert to datetime

    # Define color scale for actions
    action_color_scale = alt.Scale(domain=['BUY', 'SELL'],
                                   range=['blue', 'red'])

    # Plot action points
    action_points = alt.Chart(history_df).mark_circle(size=60).encode(
        x='date:T',
        y=alt.Y('price:Q', title='Price'),
        color=alt.Color('action:N', scale=action_color_scale),
        size='quantity:Q',
        tooltip=['date:T', 'price:Q', 'action:N', 'quantity:Q']
    ).interactive(bind_y=False)

    # Combine the line chart with action points
    final_chart = alt.layer(price_chart, action_points, title=title).properties(height=300, width=1000)

    return final_chart


def main():
    coloredlogs.install(level='DEBUG')
    switch_k_backend_device()

    # model_name = 'models/fs_model_lstm.keras'
    test_stock = 'data/predictions/fs_best.csv'  # Dataset with real and predicted values
    debug = True

    # Load the dataset
    df = pd.read_csv(test_stock)

    # Initialize the agent with the pre-trained model
    agent = Agent(initial_capital=10000, max_trade_percent=0.25, buy_threshold=0.055, sell_threshold=-0.0054, sell_high_threshold=0.1)

    history, final_assets = evaluate_model(agent, df, debug)

    print("History:", history)
    print(f"Final assets: {final_assets}") # Total value of assets at end of trading session. Cash Balance + Stock Value of the stocks owned at the end of the session (stock price of last day). Went down from 10,000 to 7,413 because 38 shares were held onto

    # Visualize the trading session
    chart = visualize(df, history, title=test_stock)
    chart.display()

if __name__ == "__main__":
    main()

2024-05-07 21:21:36 LAPTOP-CU5PMAR9 root[31172] INFO Switching to CPU for TensorFlow
227it [00:00, 11141.62it/s]            

History: [('2021-08-13', 515.92, 'BUY', 5), ('2021-08-16', 517.92, 'HOLD', 0), ('2021-08-17', 518.91, 'HOLD', 0), ('2021-08-18', 521.87, 'HOLD', 0), ('2021-08-19', 543.71, 'HOLD', 0), ('2021-08-20', 546.88, 'HOLD', 0), ('2021-08-23', 553.33, 'HOLD', 0), ('2021-08-24', 553.41, 'BUY', 3), ('2021-08-25', 547.58, 'BUY', 3), ('2021-08-26', 550.12, 'HOLD', 0), ('2021-08-27', 558.92, 'HOLD', 0), ('2021-08-30', 566.18, 'HOLD', 0), ('2021-08-31', 569.19, 'BUY', 2), ('2021-09-01', 582.07, 'BUY', 1), ('2021-09-02', 588.55, 'BUY', 1), ('2021-09-03', 590.53, 'BUY', 1), ('2021-09-07', 606.71, 'SELL', 16), ('2021-09-08', 606.05, 'HOLD', 0), ('2021-09-09', 597.54, 'HOLD', 0), ('2021-09-10', 598.72, 'HOLD', 0), ('2021-09-13', 589.29, 'HOLD', 0), ('2021-09-14', 577.76, 'HOLD', 0), ('2021-09-15', 582.87, 'HOLD', 0), ('2021-09-16', 586.5, 'HOLD', 0), ('2021-09-17', 589.35, 'HOLD', 0), ('2021-09-20', 575.43, 'HOLD', 0), ('2021-09-21', 573.14, 'HOLD', 0), ('2021-09-22', 590.65, 'HOLD', 0), ('2021-09-23', 59

alt.LayerChart(...)

Testing

In [68]:
import pandas as pd

# Load data
data = pd.read_csv('data/predictions/fs_best1.csv')  # Assuming the file is named data.csv

# Initial conditions
initial_money = 1000
money = initial_money
stocks = 0
buy_threshold = 0.01  # 1% increase
sell_threshold = -0.01  # 1% decrease

# First day buy at least 1 stock
first_day_price = df['Real'].iloc[0]
stocks = int(money / first_day_price)
money -= stocks * first_day_price
print(f'Bought {stocks} stocks at {first_day_price}, Total Stocks: {stocks}, Money left: {money}')


# Trading simulation
for i in range(len(data) - 1):
    today_real = data['Real'].iloc[i]
    today_pred = data['Predicted'].iloc[i]
    next_pred = data['Predicted'].iloc[i + 1]
    change_perc = (next_pred - today_pred) / today_pred
    print("change_perc:", change_perc)

    # Buy decision
    if change_perc > buy_threshold and money >= today_real:
        stocks_bought = money // today_real
        stocks += stocks_bought
        money -= stocks_bought * today_real
        print(f'Bought {stocks_bought} stocks at {today_real}, Total Stocks: {stocks}, Money left: {money}')

    # Sell decision
    elif change_perc < sell_threshold and stocks > 0:
        money += stocks * today_real
        print(f'Sold {stocks} stocks at {today_real}, Total Stocks: 0, Money: {money}')
        stocks = 0

# Final asset calculation
final_money = money + stocks * data['Real'].iloc[-1]
profit = final_money - initial_money
print(f'Initial money: {initial_money}')
print(f'Final money after selling stocks: {final_money}')
print(f'Profit made by the trading bot: {profit}')


Bought 1 stocks at 515.92, Total Stocks: 1, Money left: 484.08000000000004
change_perc: -0.003005719910412641
change_perc: -0.0007269680289865129
change_perc: 0.00282940677366672
change_perc: 0.007018637377362294
change_perc: 0.01444733776193647
change_perc: 0.01163962249248186
change_perc: 0.018828501840493124
change_perc: 0.020597813798676097
change_perc: 0.0029966011196570137
change_perc: 0.004127083421204859
change_perc: 0.0021343423734287983
change_perc: 0.0036557381258277926
change_perc: 0.010656640506030842
change_perc: 0.017905502523084676
change_perc: 0.01448576483812031
change_perc: 0.01222823827447236
change_perc: 0.02134558007863372
change_perc: 0.010710566188381584
change_perc: 0.005622831275240417
change_perc: 0.008925263688908126
change_perc: -0.008950229635643575
change_perc: -0.01393254395880506
Sold 1 stocks at 577.76, Total Stocks: 0, Money: 1061.8400000000001
change_perc: -0.005458540108883166
change_perc: -0.009365720735821505
change_perc: -0.0030158852997250692
ch